# Economic Dispatch modular modeling primer

This notebook is intended to show a power system modeling framework that exploits the capabilities of Julia to improve performance and allow modelers to develop modular system to analyze problems with different complexities. The aim is to replicate the practices of other communities such as Climate Change Integrated Assessment Models (IAM) that allow the integration of expertise from different fields into large scale optimization problems. An example of these efforts is the [Mimi modeling framework](https://github.com/davidanthoff/Mimi.jl). 

The approach is divided in two main portions: 

- Type definitions for power system components. (Discussed in another notebook)
- Model definitions and modifiers. 

The first version of the modeling platform is based on [JuMP](https://github.com/JuliaOpt/JuMP.jl) model modification.  Currently JuMP has some difficulties with model modification, however, in [in this post](https://discourse.julialang.org/t/mathoptinterface-and-upcoming-breaking-changes-in-jump-0-19/4874) the developers have expressed the interest in expanding the capabilities in JuMP to make the modifications easier. 

Finally, the current version of the code should be interpreted as a proof of concept considering that my expertise is power systems and not software engineering. In the future, the code can be improved with better practices. The code has been developed based on the [Economic Distpatch (ED) Problem](https://en.wikipedia.org/wiki/Economic_dispatch).

The example system for this notebook is the [Small Test Systems for Power System Economic Studies](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5589973), modified to include 2 PV-Plants and time series of load and renewable energy. The system data can also be accessed in [Matpower format](http://www.pserc.cornell.edu/matpower/docs/ref/matpower5.0/case5.html). 

## 5 - Bus system example

It is possible to store the data for the 5-bus system described in the later sections in terms of the aforementioned type structure. 

The one-line diagram of the system is as follows, where the peak load of each bus is shown: 

![5_bus_system](5bus.png)

First the nodes and system parameters are defined in terms of the system and bus types. Which eventually allows for the re-use of the same information is some other components of the system change. 

In [1]:
include("../ed_models/data_5bus.jl");

In [ ]:
include("../base_tools/build_network.jl")
using build_network
include("../ed_models/data_5bus.jl");
net = create_network(FiveBus, Branches, nodes);

## Problem creators 

The concept of the model creator is a function that takes data stored in the different structures and produces a JuMP model that can be manipulated. The process is shown un the following figure: 

![Model_creator](model_creator.png)

In the case of the ED problem, the function ``build_ed()`` takes data from the system, buses and generators and returns a JuMP model that can be solved or modified. In the five-bus case, the model can be printed and read easily.  

In [ ]:
include("../ed_models/economic_dispatch.jl")
using economic_dispatch;
include("../ed_models/renewables_ed.jl")
using renewables_ed

In [ ]:
include("../ed_models/transmission_constraints.jl")
using transmission_constraints

In [ ]:
ed_problem1 = build_ed(FiveBus, Loads, Generators)

Once the model is created and returned, it can be passed to the solving function. Inside this function is possible to define the solver and the solver parameters. For this version of the code, the solving function doesn't accept parameters for the solver. 

The solving function returns a solved model that later can be used to parse out the solution. The process returns the results as follows:

In [ ]:
ed_solution1 = solve_ed(ed_problem1);
results = results_ed(ed_solution1)

It is possible to define a modifier to add security constrains to the ED problem. When studying renewable energy integration there are many alternatives to model resources and technologies. In this regard, the objective in MEMFS is to provide what are called model modifiers, which can take a model and include the constraints or changes to the objective function to produce new model formulations. The process is as follows: 

![Model_creator](modifier.png)

Julia's capabilities to handle models as variables of a certain structure makes the process straightforward. For example, the function ```inyection_constraints()``` takes a JuMP model of ED and system data to return the modified problem. This problem then can be passed to the solving function and results functions to compare the results with respect to the solution of the base ED problem. 

In [ ]:
ed_const = inyection_constraints(ed_problem1, FiveBus, Loads, Generators, net)

In [ ]:
ed_solution2 = solve_ed(ed_const);
results = results_ed(ed_solution2)

In [ ]:
ed_re = renewables_withcurtailment(ed_const, FiveBus, PV)

In [ ]:
ed_solution3 = solve_ed(ed_re);
results = results_ed(ed_solution3)